In [ ]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm
import random

In [ ]:
with open("./ambiguous_questions.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions_pr = json.load(f)
    
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)
    
with open("./intermediate_questions.json") as f:
    intermediate_questions = json.load(f)

In [ ]:
sample_count = 0

import csv

sample_list = [["q_id", 'image_id', "ambiguous_question",'ambiguous_question_answer', "ambiguous_entity" ,"additional_question", "additional_question_answer", "entity_id"]]

stop_flag = False


total_iq = 0
total_no_aq = 0
total_yes_aq = 0

for idx, (qid, example) in tqdm(enumerate(ambiguous_questions.items())):
   
    image_id = example['imageId']
    ambiguous_question_example = ambiguous_questions_pr[qid]
    if 'irrelated_object_names' in example.keys():
        example['ambiguous_entities'] = example.pop('irrelated_object_names')
    
    
    
    unique_ambiguous_entities = set(example['ambiguous_entities'].values())
    # print(unique_ambiguous_entities)

    # if idx < 50:
    #     print(example['question'])
    #     print(example['annotations']['question'].values())
    #     print([sceneGraphs[image_id]['objects'][key]['name'] for key in example['annotations']['question'].values()])
    #     print(set(example['annotations']['question'].items()))
    
    flag = True
    for key in set(example['annotations']['question'].values()):
        entity_name = sceneGraphs[image_id]['objects'][key]['name']
        
        ambiguous_question = ambiguous_question_example['question']
        ambiguous_answer = example['answer']
        
        if entity_name in unique_ambiguous_entities and entity_name in ambiguous_question:
            
            if key in intermediate_questions[image_id].keys():
                
                intermediate_question_examples = intermediate_questions[image_id][key]
                for intermediate_question_example in intermediate_question_examples:
                    if intermediate_question_example["label"] == "positive":
                        iq = intermediate_question_example["question"]
                        ia = intermediate_question_example['answer']
            
                        sample_list.append([qid, image_id, ambiguous_question, ambiguous_answer, entity_name ,iq, ia, key])

            if key not in intermediate_questions[image_id].keys():
                flag = False
                # pprint(ambiguous_question_example)
                # print(key)
                # print(intermediate_questions[image_id].keys())
                # pprint(intermediate_questions[image_id])
                # print(entity_name)
                # print(ambiguous_question)
            

    if flag is False:
        total_no_aq +=1
    else:
        total_yes_aq += 1
        
total_iq = len(sample_list)

    
    # question_count = 0
    # ambiguous_count = 0
    # ambiguous_entity = 0
    # for object_key, intermediate_question_list in intermediate_question_objects.items():
        
    #     question_count += len(intermediate_question_list)
    #     ambiguous_count +=1
        
    #     sample_count += 1
    #     iq = intermediate_question_example["question"]
    #     answer = intermediate_question_example['answer']
    #     sample_list.append([qid, image_id, ambiguous_question['question'], iq, answer])
  
    #     if sample_count == 100:
    #         stop_flag = True
    #         break
    
    # count_list.append(question_count)
    # ambiguous_count_list.append(ambiguous_count)   
    
    # if stop_flag:
    #     break


print(total_iq)
print(total_yes_aq)
print(total_no_aq)

with open("intermediate_questions_samples.csv", "w") as f:
    writer = csv.writer(f)
    
    writer.writerows(sample_list)
    
    
    

In [ ]:
import os
from tqdm import tqdm
import json

    
with open("./intermediate_questions.json") as f:
    intermediate_questions = json.load(f)

for intermediate_question_key in tqdm(intermediate_questions.keys()):
    
    os.system(f"cp images/{intermediate_question_key}.jpg ambiguous_images/")

In [ ]:
sample_count = 0

import csv

sample_list = [["q_id", 'image_id', "original_question" ,"ambiguous_question",'ambiguous_question_answer', "ambiguous_entity" ,"additional_question", "additional_question_answer", "entity_id"]]

stop_flag = False


total_iq = 0
total_no_aq = 0
total_yes_aq = 0

count_checker = {}
random.seed(42)

for idx, (qid, example) in tqdm(enumerate(ambiguous_questions.items())):
    
    # targe image id
    image_id = example['imageId']

    # preprocessed orginal question
    ambiguous_question_example = ambiguous_questions_pr[qid]
    
    # ambiguous entities
    if 'irrelated_object_names' in example.keys():
        example['ambiguous_entities'] = example.pop('irrelated_object_names')
    
    # get names of ambiguous entities 
    unique_ambiguous_entities = set(example['ambiguous_entities'].values())
    
    if image_id not in count_checker.keys():
        count_checker[image_id] = {}
    
    flag = True
    for key in set(example['annotations']['question'].values()):
        
        # key is object index
        entity_name = sceneGraphs[image_id]['objects'][key]['name']
        
        original_question = ambiguous_question_example['original_question']
        ambiguous_question = ambiguous_question_example['question']
        ambiguous_answer = example['answer']

        if entity_name in unique_ambiguous_entities and entity_name in ambiguous_question:
            
            if key in intermediate_questions[image_id].keys():
                
                intermediate_question_examples = intermediate_questions[image_id][key]
                
                iq_idxs = list(range(len(intermediate_question_examples)))
                if key in count_checker[image_id]:
                    exist_idxs = count_checker[image_id]
                    iq_idxs = [iq_idx for iq_idx in iq_idxs if iq_idx not in exist_idxs]
                    
                random.shuffle(iq_idxs)
    
                for iq_idx in iq_idxs:
                    intermediate_question_example = intermediate_question_examples[iq_idx]
                    if intermediate_question_example["label"] == "positive":
                        iq = intermediate_question_example["question"]
                        ia = intermediate_question_example['answer']

                        if key not in count_checker[image_id].keys():
                            count_checker[image_id] = {key : [iq_idx]}
                        else:
                            if len(count_checker[image_id][key]) < 4 and iq_idx not in count_checker[image_id][key]:
                                count_checker[image_id][key].append(iq_idx)
                                sample_list.append([qid, image_id, original_question ,ambiguous_question, ambiguous_answer, entity_name ,iq, ia, key])

            if key not in intermediate_questions[image_id].keys():
                flag = False
            

    if flag is False:
        total_no_aq +=1
    else:
        total_yes_aq += 1
        
total_iq = len(sample_list)


print(total_iq)
print(total_yes_aq)
print(total_no_aq)

with open("intermediate_questions_samples_25000.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(sample_list[:25001])
    

In [ ]:
import pandas as pd
import math
from pprint import pprint 
with open("IQG.csv", 'r') as f:
    df = pd.read_csv(f, )
    
# pprint(df.iloc[:5])


new_df = df.drop(labels=['url-1', 'url', '작업차수'], axis=1)

wrong_list = []
for idx, row in new_df.iterrows():
    
    if type(row['비고']) == str:
        wrong_list.append(idx)

clean_df = new_df.drop(labels=wrong_list, axis=0)

print(len(clean_df))